![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_XlmRoBertaForZeroShotClassification.ipynb)

## Import ONNX XlmRoBertaForZeroShotClassification  models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `XlmRoBertaForZeroShotClassification ` is only available since in `Spark NLP 5.2.4` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import BERT models trained/fine-tuned for zero shot classification via `XlmRoBertaForZeroShotClassification` or `TFXlmRoBertaForZeroShotClassification`. These models are usually under `Zero-Shot Classification` category and have `bert` in their labels
- Reference: [TFXlmRoBertaForZeroShotClassification](https://huggingface.co/docs/transformers/en/model_doc/xlm-roberta#transformers.TFXLMRobertaForSequenceClassification)
- Some [example models](https://huggingface.co/models?pipeline_tag=zero-shot-classification&sort=downloads&search=xlm-roberta)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.52.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.52.3 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [symanto/xlm-roberta-base-snli-mnli-anli-xnli](https://huggingface.co/symanto/xlm-roberta-base-snli-mnli-anli-xnli) model from HuggingFace as an example and load it as a `ORTModelForSequenceClassification`, representing an ONNX model.
- In addition to the XLM-RoBERTa model, we also need to save the `XLMRobertaTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import XLMRobertaTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification

MODEL_NAME = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

('onnx_models/symanto/xlm-roberta-base-snli-mnli-anli-xnli/tokenizer_config.json',
 'onnx_models/symanto/xlm-roberta-base-snli-mnli-anli-xnli/special_tokens_map.json',
 'onnx_models/symanto/xlm-roberta-base-snli-mnli-anli-xnli/sentencepiece.bpe.model',
 'onnx_models/symanto/xlm-roberta-base-snli-mnli-anli-xnli/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 1091400
-rw-r--r-- 1 root root        854 Jun 26 17:11 config.json
-rw-r--r-- 1 root root 1112503258 Jun 26 17:11 model.onnx
-rw-r--r-- 1 root root    5069051 Jun 26 17:11 sentencepiece.bpe.model
-rw-r--r-- 1 root root        964 Jun 26 17:11 special_tokens_map.json
-rw-r--r-- 1 root root       1203 Jun 26 17:11 tokenizer_config.json


- We need to move `sentencepiece.bpe.model` from the tokenizer to `assets` folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [4]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/sentencepiece.bpe.model {EXPORT_PATH}/assets

In [6]:
labels = ort_model.config.id2label
labels = [value for key,value in sorted(labels.items(), reverse=False)]

with open(EXPORT_PATH + '/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

In [7]:
!ls -lR {EXPORT_PATH}

onnx_models/symanto/xlm-roberta-base-snli-mnli-anli-xnli:
total 1086452
drwxr-xr-x 2 root root       4096 Jun 26 17:12 assets
-rw-r--r-- 1 root root        854 Jun 26 17:11 config.json
-rw-r--r-- 1 root root 1112503258 Jun 26 17:11 model.onnx
-rw-r--r-- 1 root root        964 Jun 26 17:11 special_tokens_map.json
-rw-r--r-- 1 root root       1203 Jun 26 17:11 tokenizer_config.json

onnx_models/symanto/xlm-roberta-base-snli-mnli-anli-xnli/assets:
total 4956
-rw-r--r-- 1 root root      32 Jun 26 17:12 labels.txt
-rw-r--r-- 1 root root 5069051 Jun 26 17:11 sentencepiece.bpe.model


Voila! We have our `sentencepiece.bpe.model` and `labels.txt` inside assets directory

## Import and Save XlmRoBertaForZeroShotClassification in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [8]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 14.5 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [9]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `XlmRoBertaForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `XlmRoBertaForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [10]:
from sparknlp.annotator import XlmRoBertaForZeroShotClassification

zero_shot_classifier = XlmRoBertaForZeroShotClassification.loadSavedModel(
    EXPORT_PATH,
    spark
  )\
  .setInputCols(["document", "token"]) \
  .setOutputCol("class") \
  .setCandidateLabels(["urgent", "mobile", "travel", "movie", "music", "sport", "weather", "technology"])

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [12]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your XlmRoBertaForZeroShotClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [13]:
! ls -l {MODEL_NAME}_spark_nlp_onnx

total 1091560
drwxr-xr-x 3 root root       4096 Jun 26 17:16 fields
drwxr-xr-x 2 root root       4096 Jun 26 17:16 metadata
-rw-r--r-- 1 root root 1112673147 Jun 26 17:16 xlmroberta_classification_onnx
-rw-r--r-- 1 root root    5069051 Jun 26 17:16 xlmroberta_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny XlmRoBertaForZeroShotClassification model 😊

In [14]:
zero_shot_classifier_loaded = XlmRoBertaForZeroShotClassification.load("./{}_spark_nlp_onnx".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [15]:
zero_shot_classifier_loaded.getClasses()

['ENTAILMENT', 'NEUTRAL', 'CONTRADICTION']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [16]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

texts = spark.createDataFrame([
    ["I have a problem with my iPhone that needs to be resolved asap!!"],
    ["Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."],
    ["I have a phone and I love it!"],
    ["I really want to visit Germany and I am planning to go there next year."],
    ["Let's watch some movies tonight! I am in the mood for a horror movie."],
    ["Have you watched the match yesterday? It was a great game!"],
    ["We need to hurry up and get to the airport. We are going to miss our flight!"]
], ["text"])

model = pipeline.fit(texts)
result = model.transform(texts)

result.selectExpr("text", "class.result[0] as predicted_label").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------+---------------+
|text                                                                                                          |predicted_label|
+--------------------------------------------------------------------------------------------------------------+---------------+
|I have a problem with my iPhone that needs to be resolved asap!!                                              |urgent         |
|Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app.|technology     |
|I have a phone and I love it!                                                                                 |mobile         |
|I really want to visit Germany and I am planning to go there next year.                                       |travel         |
|Let's watch some movies tonight! I am in the mood for a horror movie.                           

That's it! You can now go wild and use hundreds of `XlmRoBertaForZeroShotClassification` models from HuggingFace 🤗 in Spark NLP 🚀
